# Embedding Model

SGLang supports embedding models in the same way as completion models. Here are some example models:

- [intfloat/e5-mistral-7b-instruct](https://huggingface.co/intfloat/e5-mistral-7b-instruct)
- [Alibaba-NLP/gte-Qwen2-7B-instruct](https://huggingface.co/Alibaba-NLP/gte-Qwen2-7B-instruct)


## Launch A Server

The following code is equivalent to running this in the shell:
```bash
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --port 30010 --host 0.0.0.0 --is-embedding
```

Remember to add `--is-embedding` to the command.

In [1]:
from sglang.utils import lauch_sglang_server, wait_for_server, terminate_process, highlight_text

embedding_process = lauch_sglang_server(
    """
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --port 30010 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server("http://localhost:30010")

highlight_text("Embedding server is ready. Proceeding with the next steps.")

[2024-10-28 09:15:25] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-7B-instruct', chat_template=None, is_embedding=True, host='0.0.0.0', port=30010, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=237179517, constrained_json_whitespace_pattern=None, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, api_key=None, file_storage_pth='SGLang_storage', enable_cache_report=False, watchdog_timeout=600, dp_size=1, load_balance_method='round_robin', dist_init_addr=None, nnodes=1, node_rank=0,

[2024-10-28 09:15:40 TP0] Init torch distributed begin.


[2024-10-28 09:15:41 TP0] Load weight begin. avail mem=78.59 GB


[2024-10-28 09:15:41 TP0] lm_eval is not installed, GPTQ may not be usable


INFO 10-28 09:15:41 weight_utils.py:243] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards:  71% Completed | 5/7 [00:09<00:03,  1.97s/it]


Loading safetensors checkpoint shards:  86% Completed | 6/7 [00:11<00:01,  1.81s/it]


Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:12<00:00,  1.52s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:12<00:00,  1.74s/it]

[2024-10-28 09:15:54 TP0] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=64.18 GB
[2024-10-28 09:15:54 TP0] Memory pool end. avail mem=7.43 GB


[2024-10-28 09:15:54 TP0] max_total_num_tokens=1025173, max_prefill_tokens=16384, max_running_requests=4005, context_len=131072


[2024-10-28 09:15:54] INFO:     Started server process [509328]
[2024-10-28 09:15:54] INFO:     Waiting for application startup.
[2024-10-28 09:15:54] INFO:     Application startup complete.
[2024-10-28 09:15:54] INFO:     Uvicorn running on http://0.0.0.0:30010 (Press CTRL+C to quit)
[2024-10-28 09:15:54] INFO:     127.0.0.1:59258 - "GET /v1/models HTTP/1.1" 200 OK


## Use Curl

In [2]:
import subprocess, json

text = "Once upon a time"

curl_text = f"""curl -s http://localhost:30010/v1/embeddings \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer None" \
  -d '{{"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "input": "{text}"}}'"""

text_embedding = json.loads(subprocess.check_output(curl_text, shell=True))["data"][0][
    "embedding"
]

highlight_text(f"Text embedding (first 10): {text_embedding[:10]}")

[2024-10-28 09:15:55 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-10-28 09:15:55] INFO:     127.0.0.1:59280 - "GET /get_model_info HTTP/1.1" 200 OK


[2024-10-28 09:15:56 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 1, #queue-req: 0
[2024-10-28 09:15:56] INFO:     127.0.0.1:59274 - "POST /v1/embeddings HTTP/1.1" 200 OK


## Using OpenAI Compatible API

In [3]:
import openai

client = openai.Client(base_url="http://127.0.0.1:30010/v1", api_key="None")

# Text embedding example
response = client.embeddings.create(
    model="Alibaba-NLP/gte-Qwen2-7B-instruct",
    input=text,
)

embedding = response.data[0].embedding[:10]
highlight_text(f"Text embedding (first 10): {embedding}")

[2024-10-28 09:15:56] INFO:     127.0.0.1:59290 - "POST /encode HTTP/1.1" 200 OK
[2024-10-28 09:15:56] The server is fired up and ready to roll!


[2024-10-28 09:15:56 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 3, cache hit rate: 21.43%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-28 09:15:56] INFO:     127.0.0.1:59300 - "POST /v1/embeddings HTTP/1.1" 200 OK


## Using Input IDs

SGLang also supports `input_ids` as input to get the embedding.

In [4]:
import json
import os
from transformers import AutoTokenizer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

tokenizer = AutoTokenizer.from_pretrained("Alibaba-NLP/gte-Qwen2-7B-instruct")
input_ids = tokenizer.encode(text)

curl_ids = f"""curl -s http://localhost:30010/v1/embeddings \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer None" \
  -d '{{"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "input": {json.dumps(input_ids)}}}'"""

input_ids_embedding = json.loads(subprocess.check_output(curl_ids, shell=True))["data"][
    0
]["embedding"]

highlight_text(f"Input IDs embedding (first 10): {input_ids_embedding[:10]}")

[2024-10-28 09:16:02 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 3, cache hit rate: 33.33%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-28 09:16:02] INFO:     127.0.0.1:59034 - "POST /v1/embeddings HTTP/1.1" 200 OK


In [5]:
terminate_process(embedding_process)

[2024-10-28 09:16:02] INFO:     Shutting down
[2024-10-28 09:16:02] INFO:     Waiting for application shutdown.
[2024-10-28 09:16:02] INFO:     Application shutdown complete.
[2024-10-28 09:16:02] INFO:     Finished server process [509328]
